**Import Libaraies**

In [11]:
# Importing libraries 
import numpy as np 
import tensorflow 
from tensorflow import keras 
import matplotlib as plt 
import seaborn 
import glob 
import os
from os import listdir 

from keras.preprocessing.image import ImageDataGenerator

import skimage 
from skimage.io import imread_collection
from skimage.io import imshow_collection
from skimage.io import concatenate_images 
from matplotlib import pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import Sequential 

from PIL import Image

In [12]:
for dirname, _, filenames in os.walk('C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\'):
    print (dirname)

C:\Users\0MEN\Desktop\sub\resizedto224\
C:\Users\0MEN\Desktop\sub\resizedto224\pred
C:\Users\0MEN\Desktop\sub\resizedto224\Test
C:\Users\0MEN\Desktop\sub\resizedto224\Test\actinic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Test\basal cell carcinoma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\dermatofibroma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\melanoma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\nevus
C:\Users\0MEN\Desktop\sub\resizedto224\Test\pigmented benign keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Test\seborrheic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Train
C:\Users\0MEN\Desktop\sub\resizedto224\Train\actinic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Train\basal cell carcinoma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\dermatofibroma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\melanoma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\nevus
C:\Users\0MEN\Desktop\sub\resizedto224\Train\pigmented benign keratosis
C:\Users\0MEN\Desktop\sub\resizedto2

In [13]:
# Importing the data and preprocessing 
from keras.preprocessing.image import ImageDataGenerator 


# set the directory based on the folder where you saved dataset
train_data_dir = "C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\Train\\"
test_data_dir = "C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\Test\\"
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_data_dir,
                                                 shuffle=True,
                                                 target_size=(227,227), 
                                                 subset="training",
                                                 class_mode='categorical')

val_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_data_dir,
                                                 shuffle=True,
                                                 target_size=(227,227), 
                                                 subset="validation",
                                                 class_mode='categorical')
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(test_data_dir,
                                       target_size=(227,227), batch_size=32, shuffle=False)


print(train_data.classes)
print(val_data.classes)
print(train_data.class_indices)
print(test_data.classes)
class_names = np.unique(train_data.classes)

Found 1538 images belonging to 7 classes.
Found 381 images belonging to 7 classes.
Found 99 images belonging to 7 classes.
[0 0 0 ... 6 6 6]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6]
{'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 

In [14]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(7, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_5 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_6 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 384)      

In [15]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(train_data, epochs=25, batch_size=32, validation_data = val_data)

Epoch 1/25
49/49 [==============================] - 6s 109ms/step - loss: 17.6506 - accuracy: 0.3049 - val_loss: 100.8563 - val_accuracy: 0.2283
Epoch 2/25
49/49 [==============================] - 5s 105ms/step - loss: 3.6213 - accuracy: 0.3882 - val_loss: 2.3571 - val_accuracy: 0.2021
Epoch 3/25
49/49 [==============================] - 5s 105ms/step - loss: 2.1851 - accuracy: 0.4220 - val_loss: 3.7701 - val_accuracy: 0.2126
Epoch 4/25
49/49 [==============================] - 5s 105ms/step - loss: 2.1075 - accuracy: 0.4454 - val_loss: 2.0466 - val_accuracy: 0.2730
Epoch 5/25
49/49 [==============================] - 5s 104ms/step - loss: 1.7759 - accuracy: 0.4746 - val_loss: 1.9872 - val_accuracy: 0.3018
Epoch 6/25
49/49 [==============================] - 5s 107ms/step - loss: 1.9275 - accuracy: 0.4545 - val_loss: 1.9016 - val_accuracy: 0.2677
Epoch 7/25
49/49 [==============================] - 5s 105ms/step - loss: 1.7670 - accuracy: 0.4564 - val_loss: 1.9212 - val_accuracy: 0.3176
Epo

In [17]:
test_loss, test_acc = model.evaluate((test_data))

4/4 [==============================] - 0s 31ms/step - loss: 2.6052 - accuracy: 0.2626


In [18]:
model.save('C:\\Users\\0MEN\\Desktop\\sub\\alexnet\\intel_alexnet_dec21.h5')